In [ ]:

import sys
%matplotlib inline

from data_models.parameters import arl_path

results_dir = arl_path('test_results')

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u

from matplotlib import pyplot as plt

from data_models.polarisation import PolarisationFrame
from data_models.memory_data_models import Skycomponent, SkyModel

from wrappers.serial.visibility.base import create_blockvisibility
from wrappers.serial.image.operations import show_image, qa_image
from wrappers.serial.simulation.testing_support import create_named_configuration, simulate_pointingtable
from wrappers.serial.imaging.primary_beams import create_vp, create_pb
from wrappers.serial.skycomponent.operations import apply_beam_to_skycomponent
from wrappers.serial.imaging.base import create_image_from_visibility, advise_wide_field
from processing_components.calibration.pointing import create_pointingtable_from_blockvisibility
from processing_components.simulation.pointing import create_gaintable_from_pointingtable
from wrappers.arlexecute.visibility.base import copy_visibility


from wrappers.arlexecute.visibility.coalesce import convert_blockvisibility_to_visibility

from workflows.arlexecute.skymodel.skymodel_arlexecute import predict_skymodel_list_arlexecute_workflow
from workflows.arlexecute.imaging.imaging_arlexecute import invert_list_arlexecute_workflow

from wrappers.arlexecute.execution_support.arlexecute import arlexecute

import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))


In [ ]:
arlexecute.set_client(use_dask=True)

In [ ]:
nfreqwin = 1
ntimes = 1
rmax = 1000.0
if nfreqwin > 1:
    frequency = numpy.linspace(0.8e9, 1.2e9, nfreqwin)
    channel_bandwidth = numpy.array(nfreqwin * [frequency[1] - frequency[0]])
else:
    frequency = [1e9]
    channel_bandwidth = [0.4e9]
times = numpy.linspace(-numpy.pi / 4.0, numpy.pi / 4.0, ntimes)

phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox='J2000')
midcore = create_named_configuration('MIDR5', rmax=rmax)

block_vis = create_blockvisibility(midcore, times, frequency=frequency,
                                   channel_bandwidth=channel_bandwidth, weight=1.0, phasecentre=phasecentre,
                                   polarisation_frame=PolarisationFrame("stokesI"), zerow=True)


In [ ]:
wprojection_planes=1
vis = convert_blockvisibility_to_visibility(block_vis)
advice=advise_wide_field(vis, guard_band_image=5.0, delA=0.02, 
                         wprojection_planes=wprojection_planes)

vis_slices = advice['vis_slices']
npixel=16384
cellsize=advice['cellsize']
pb_cellsize=advice['primary_beam_fov']/npixel

In [ ]:
from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'
doplot=False

In [ ]:
original_component = [Skycomponent(flux=[[1.0]], direction=phasecentre, 
                                  frequency=frequency, polarisation_frame=PolarisationFrame('stokesI'))]
vp = create_image_from_visibility(block_vis, npixel=npixel, frequency=frequency,
                                    nchan=nfreqwin, cellsize=pb_cellsize, phasecentre=phasecentre)
model = create_image_from_visibility(block_vis, npixel=512, frequency=frequency,
                                    nchan=nfreqwin, cellsize=cellsize, phasecentre=phasecentre)
if doplot:
    pb = create_pb(vp, 'MID', pointingcentre=pb_direction)
    show_image(pb)
    
offset = [0.595, 0.0]
pb_direction = SkyCoord(ra=(+15.0 + offset[0]) * u.deg, 
                        dec=(-45.0 + offset[1]) * u.deg, frame='icrs', equinox='J2000')

vp = create_vp(vp, 'MID', pointingcentre=pb_direction)
pt = create_pointingtable_from_blockvisibility(block_vis, vp)

no_error_pt = simulate_pointingtable(pt, 0.0, 0.0)
no_error_gt = create_gaintable_from_pointingtable(block_vis, original_component, no_error_pt, vp)

no_error_sm=SkyModel(components=original_component, gaintable=no_error_gt[0])
no_error_vis = copy_visibility(vis)
no_error_vis = predict_skymodel_list_arlexecute_workflow(no_error_vis, [no_error_sm], context='2d', docal=True)
no_error_vis=arlexecute.compute(no_error_vis, sync=True)[0]


In [ ]:
print(numpy.average(no_error_vis.vis))

Generate the model from the GLEAM catalog, including application of the primary beam.

In [ ]:

fwhm = 1.0
pes = [1.0, 3.0, 10.0, 30.0, 100.0, 300.0, 1000.0]
qas = []
for pe in pes:
    pointing_error = pe * numpy.pi/(3600.0*180.0)
    static_pointing_error = 0.0*numpy.pi/(3600.0*180.0)
    error_pt = simulate_pointingtable(pt, pointing_error=pointing_error, static_pointing_error=static_pointing_error)
    error_gt = create_gaintable_from_pointingtable(block_vis, original_component, error_pt, vp)

    error_sm=SkyModel(components=original_component, gaintable=error_gt[0])

    error_vis = copy_visibility(vis)
    error_vis = predict_skymodel_list_arlexecute_workflow(error_vis, [error_sm], context='2d', docal=True)
    error_vis=arlexecute.compute(error_vis, sync=True)[0]

    error_vis.data['vis']-=no_error_vis.data['vis']
    
    dirty = invert_list_arlexecute_workflow([error_vis], [model], '2d', padding=4)
    dirty, sumwt = arlexecute.compute(dirty, sync=True)[0]
    bpix=129
    epix=384
    print(bpix,epix)
    print(dirty.data.shape)

    dirty.data = dirty.data[:,:,bpix:epix,bpix:epix]
    qa=qa_image(dirty)
    print(qa)
    qas.append(qa)
    if True:
        show_image(dirty)
        plt.show()


In [ ]:
plt.clf()
for field in ['maxabs', 'rms', 'medianabs']:
    plt.loglog(pes, [q.data[field] for q in qas], '.', label=field)
plt.xlabel('Pointing error (arcsec)')
plt.ylabel('Error (Jy)')
plt.title('Error for 1Jy point source at half power point of MID primary beam')
plt.legend()
plt.show()